In [1]:
# 
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np


ImportError: /lib64/libc.so.6: version `GLIBC_2.14' not found (required by /home/nm3191/anaconda3/envs/deeplearning/lib/python2.7/site-packages/cv2.so)

In [ ]:
import os
from matplotlib import pyplot as plt
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
PATH = os.getcwd()

data_path = PATH + '/testing augmentor'
print(data_path)
img_data_list=[]

num_images = input("How many examples do you wish to create? : ")
data_dir_list = os.listdir(data_path)
for dataset in data_dir_list:
    #print(dataset)
    img_path1 = os.listdir(data_path+'/'+dataset)
    #print(img_path1)
    #img_list=os.listdir(img_path)
    num_images_per_sample = int(num_images)/len(img_path1)

    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))

    for img1 in img_path1:
        #print(img1)
        img_path = data_path + "/" + dataset + '/'+ img1
        #print(img_path)
        img = image.load_img(img_path, target_size=(128, 128))

        x = image.img_to_array(img)
        x = x*255
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        #x = x*255
        z = x[0]
        #print(z[:,:,0].shape)
        x[0,:,:,0] = z[:,:,0]/(z[:,:,0].max())*255
        x[0,:,:,1] = z[:,:,1]/(z[:,:,1].max())*255
        x[0,:,:,2] = z[:,:,2]/(z[:,:,2].max())*255
        print('Input image shape:', x.shape)
        for i in range(int(num_images_per_sample)):
            img_data_list.append(x)



img_data = np.array(img_data_list)

#img_data = img_data.astype('float32')

print (img_data.shape)

img_data=np.rollaxis(img_data,1,0)

print (img_data.shape)

img_data=img_data[0]

print (img_data.shape)

In [ ]:
import cv2
for i in range(int(num_images)):
    imgName = "Image"+str(i)+".jpg"
    cv2.imwrite(imgName, img_data[i])

In [ ]:
img_data = img_data.astype(np.uint8)
sometimes = lambda aug: iaa.Sometimes(0.4, aug)

# Define our sequence of augmentation steps that will be applied to every image
# All augmenters with per_channel=0.5 will sample one value _per image_
# in 50% of all cases. In all other cases they will sample new values
# _per channel_.
seq = iaa.Sequential(
    [
        # apply the following augmenters to most images
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.1), # vertically flip 20% of all images
        # crop images by -5% to 10% of their height/width
        sometimes(iaa.CropAndPad(
            percent=(-0.05, 0.1),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
        )),
        sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # translate by -20 to +20 percent (per axis)
            #rotate=(-45, 45), # rotate by -45 to +45 degrees
            shear=(-16, 16), # shear by -16 to +16 degrees
            order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            #mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )),
        # execute 0 to 5 of the following (less important) augmenters per image
        # don't execute all of them, as that would often be way too strong
        iaa.SomeOf((0, 5),
            [
                #sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
                sometimes(iaa.OneOf([
                    iaa.GaussianBlur((0, 1.5)), # blur images with a sigma between 0 and 3.0
                    iaa.AverageBlur(k=(2, 4)), # blur image using local means with kernel sizes between 2 and 7
                    iaa.MedianBlur(k=(3, 5)), # blur image using local medians with kernel sizes between 2 and 7
                ])),
                iaa.Sharpen(alpha=(0, 0.5), lightness=(0.75, 1.5)), # sharpen images
                #iaa.Emboss(alpha=(0, 1.0), strength=(0, 0.5)), # emboss images
                # search either for all edges or for directed edges,
                # blend the result with the original image using a blobby mask
                iaa.OneOf([iaa.SimplexNoiseAlpha(iaa.OneOf([
                                iaa.EdgeDetect(alpha=(0.5, 1.0)),
                                iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                                ])),
                            iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
                ]),# add gaussian noise to images
                iaa.OneOf([
                    iaa.Dropout((0.01, 0.05), per_channel=0.2), # randomly remove up to 10% of the pixels
                    iaa.CoarseDropout((0.01, 0.04), size_percent=(0.02, 0.05), per_channel=0.2),
                ]),
                #iaa.Invert(0.05, per_channel=True), # invert color channels
                iaa.Add((-10, 10), per_channel=0.3), # change brightness of images (by -10 to 10 of original value)
                iaa.AddToHueAndSaturation((-15, 15)), # change hue and saturation
                # either change the brightness of the whole image (sometimes
                # per channel) or change the brightness of subareas
                #iaa.OneOf([
                #    iaa.Multiply((0.5, 1.5), per_channel=0.5),
                #    iaa.FrequencyNoiseAlpha(
                #        exponent=(-4, 0),
                #        first=iaa.Multiply((0.5, 1.5), per_channel=True),
                #        second=iaa.ContrastNormalization((0.5, 2.0))
                #    )
                #]),
                iaa.ContrastNormalization((0.75, 1.25), per_channel=0.5), # improve or worsen the contrast
                iaa.Grayscale(alpha=(0.6, 1.0)),
                sometimes(iaa.ElasticTransformation(alpha=(0.5, 1.5), sigma=0.25)), # move pixels locally around (with random strengths)
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.03))), # sometimes move parts of the image around
                sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.05)))
            ],
            random_order=True
        )
    ],
    random_order=True
)

images_aug = seq.augment_images(img_data)

In [ ]:
import cv2
for j in range(len(data_dir_list)):
    for i in range(int(num_images)):
        imgName = "data" + "/" + data_dir_list[j] + "/Image"+str(i)+".jpg"
        cv2.imwrite(imgName, images_aug[i + 50*j])

#img = images_aug[0]
#cv2.imshow('ImageWindow', img)
#cv2.waitKey()